*version 190910.0*

# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment.

In addition, I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

## Notes

1. Partial marks may be awarded if the answer is incorrect, or removed even if the answer is correct! Within each function show your understanding of course concepts such as regex, `groupby()`, `apply()`, etc. as appropriate to the question. If you unnecessarily hand code data in your functions instead of using pandas cleaning mechanisms you may be docked grades.
2. Do not including data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
3. I highly suggest that you first tackle the correlation questions, as they are all similar and worth the majority of grades for this assignment. This is by design!
4. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
5. This assignment **is not autograded**. Go to the course shell and upload your .ipynb file there as assignment4.ipynb.
6. You can earn up to 100% on this assignment by creating correlations for a single year (2016) for which there is population data. You can earn up to 105% on this assignment by going further, and creating correlations for multiple years (e.g. across the years of data we have on the sports), but it's expected that there would be a short discussion of the limitations of your approach as far as interpretation goes. Finally, if you find new population data that covers a broader set of years and integrate it into a solution across these years and sports and provide that interpretation you can earn up to 110% on the assignment.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns = {'Population (2016 est.)[8]': 'Population'}, inplace = True)

In [2]:
#deal with camel case
def camel_case_split(str): 
  
    return re.findall(r'[\w\s]+?(?:[a-z])(?=[A-Z]|$)', str)

#regex cleaning
cities.replace(to_replace=r'^\[.+', value=np.nan, regex=True, inplace=True)
cities.replace(to_replace=r'\[.+', value='', regex=True, inplace=True)
cities.replace('—', np.nan, inplace=True)

#fix camel case multi team cities
cities['NFL'] = cities['NFL'].astype(str)
cities['NFL'] = cities['NFL'].apply(camel_case_split)
cities['MLB'] = cities['MLB'].astype(str)
cities['MLB'] = cities['MLB'].apply(camel_case_split)
cities['NBA'] = cities['NBA'].astype(str)
cities['NBA'] = cities['NBA'].apply(camel_case_split)
cities['NHL'] = cities['NHL'].astype(str)
cities['NHL'] = cities['NHL'].apply(camel_case_split)

nfl_cities = cities.explode('NFL')
mlb_cities = cities.explode('MLB')
nhl_cities = cities.explode('NHL')
nba_cities = cities.explode('NBA')

In [3]:
nhl_dictionary = {
    'Anaheim Ducks': 'Ducks',
    'Arizona Coyotes': 'Coyotes',
    'Boston Bruins': 'Bruins',
    'Buffalo Sabres': 'Sabres',
    'Calgary Flames': 'Flames',
    'Carolina Hurricanes': 'Hurricanes',
    'Chicago Blackhawks': 'Blackhawks',
    'Colorado Avalanche': 'Avalanche',
    'Columbus Blue Jackets': 'Blue Jackets',
    'Dallas Stars': 'Stars',
    'Detroit Red Wings': 'Red Wings',
    'Edmonton Oilers': 'Oilers',
    'Florida Panthers': 'Panthers',
    'Los Angeles Kings': 'Kings',
    'Minnesota Wild': 'Wild',
    'Montreal Canadiens': 'Canadiens',
    'Nashville Predators': 'Predators',
    'New Jersey Devils': 'Devils',
    'New York Islanders': 'Islanders',
    'New York Rangers': 'Rangers',
    'Ottawa Senators': 'Senators',
    'Philadelphia Flyers': 'Flyers',
    'Pittsburgh Penguins': 'Penguins',
    'San Jose Sharks': 'Sharks',
    'St. Louis Blues': 'Blues',
    'Tampa Bay Lightning': 'Lightning',
    'Toronto Maple Leafs': 'Maple Leads',
    'Vancouver Canucks': 'Canucks',
    'Vegas Golden Knights': 'Golden Knights',
    'Washington Capitals': 'Capitals',
    'Winnipeg Jets': 'Jets',
    }

mlb_dictionary = {
    'Atlanta Braves' : 'Braves',
    'Miami Marlins' : 'Marlins',
    'New York Mets' : 'Mets',
    'Philadelphia Phillies' : 'Phillies',
    'Washington Nationals' : 'Nationals',
    'Chicago Cubs' : 'Cubs',
    'Cincinnati Reds' : 'Reds',
    'Milwaukee Brewers' : 'Brewers',
    'Pittsburgh Pirates' : 'Pirates',
    'St. Louis Cardinals' : 'Cardinals',
    'Arizona Diamondbacks' : 'Diamondbacks',
    'Colorado Rockies' : 'Rockies',
    'Los Angeles Dodgers': 'Dodgers',
    'San Diego Padres' : 'Padres',
    'San Francisco Giants' : 'Giants',
    'Baltimore Orioles' : 'Orioles',
    'Boston Red Sox' : 'Red Sox',
    'New York Yankees' : 'Yankees',
    'Tampa Bay Rays' : 'Rays',
    'Toronto Blue Jays' : 'Blue Jays',
    'Chicago White Sox' : 'White Sox',
    'Cleveland Indians' : 'Indians',
    'Detroit Tigers' : 'Tigers',
    'Kansas City Royals' : 'Royals',
    'Minnesota Twins' : 'Twins',
    'Houston Astros' : 'Astros',
    'Los Angeles Angels of Anaheim' : 'Angels',
    'Los Angeles Angels': 'Angels',
    'Oakland Athletics' : 'Athletics',
    'Seattle Mariners' : 'Mariners',
    'Texas Rangers' : 'Rangers'
    }
# ran into some very strange unicode characters that would only show up using loc/iloc,
# couldn't get regex to work on them

nba_dictionary = {
    'Atlanta Hawks\xa0': 'Hawks',
    'Boston Celtics\xa0': 'Celtics',
    'Brooklyn Nets\xa0': 'Nets',
    'Charlotte Hornets\xa0': 'Hornets',
    'Chicago Bulls\xa0': 'Bulls',
    'Cleveland Cavaliers\xa0': 'Cavaliers',
    'Dallas Mavericks\xa0': 'Mavericks',
    'Denver Nuggets\xa0': 'Nuggets',
    'Detroit Pistons\xa0': 'Pistons',
    'Golden State Warriors\xa0': 'Warriors',
    'Houston Rockets\xa0': 'Rockets',
    'Indiana Pacers\xa0': 'Pacers',
    'Los Angeles Clippers\xa0': 'Clippers',
    'Los Angeles Lakers\xa0': 'Lakers',
    'Memphis Grizzlies\xa0': 'Grizzlies',
    'Miami Heat\xa0': 'Heat',
    'Milwaukee Bucks\xa0': 'Bucks',
    'Minnesota Timberwolves\xa0': 'Timberwolves',
    'New Orleans Pelicans\xa0': 'Pelicans',
    'New York Knicks\xa0': 'Knicks',
    'Oklahoma City Thunder\xa0': 'Thunder',
    'Orlando Magic\xa0': 'Magic',
    'Philadelphia 76ers\xa0': '76ers',
    'Phoenix Suns\xa0': 'Suns',
    'Portland Trail Blazers\xa0': 'Blazers',
    'Sacramento Kings\xa0': 'Kings',
    'San Antonio Spurs\xa0': 'Spurs',
    'Toronto Raptors\xa0': 'Raptors',
    'Utah Jazz\xa0': 'Jazz',
    'Washington Wizards\xa0': 'Wizards',
    }

nfl_dictionary = {
    'Arizona Cardinals': 'Cardinals',
    'Atlanta Falcons': 'Falcons',
    'Baltimore Ravens': 'Ravens',
    'Buffalo Bills': 'Bills',
    'Carolina Panthers': 'Panthers',
    'Chicago Bears': 'Bears',
    'Cincinnati Bengals': 'Bengals',
    'Cleveland Browns': 'Browns',
    'Dallas Cowboys': 'Cowboys',
    'Denver Broncos': 'Broncos',
    'Detroit Lions': 'Lions',
    'Green Bay Packers': 'Packers',
    'Houston Texans': 'Texans',
    'Indianapolis Colts': 'Colts',
    'Jacksonville Jaguars': 'Jaguars',
    'Kansas City Chiefs': 'Chiefs',
    'Los Angeles Rams': 'Rams',
    'Miami Dolphins': 'Dolphins',
    'Minnesota Vikings': 'Vikings',
    'New England Patriots': 'Patriots',
    'New Orleans Saints': 'Saints',
    'New York Giants': 'Giants',
    'New York Jets': 'Jets',
    'Oakland Raiders': 'Raiders',
    'Philadelphia Eagles': 'Eagles',
    'Pittsburgh Steelers': 'Steelers',
    'San Diego Chargers': 'Chargers',
    'Los Angeles Chargers': 'Chargers',
    'San Francisco 49ers': '49ers',
    'Seattle Seahawks': 'Seahawks',
    'Tampa Bay Buccaneers': 'Buccaneers',
    'Tennessee Titans': 'Titans',
    'Washington Redskins': 'Redskins',
    }

#-----------------------------------------------------------------------------------------------------------------
#clean nhl data set
nhl_df.drop(['GP', 'OL', 'PTS', 'PTS%', 'GF', 'GA', 'SRS', 'SOS', 'RPt%', 'ROW', 'League'], axis=1, inplace=True)
#need to regex team names
nhl_df.replace(to_replace=r'[^a-zA-z\s0-9]', value='', regex=True, inplace=True)
nhl_df.set_index('team', inplace=True)
nhl_df.drop(['Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division'], inplace=True)
#map dict
nhl_df.reset_index(inplace=True)
nhl_df.replace(nhl_dictionary, inplace=True)

#-----------------------------------------------------------------------------------------------------------------
#clean nfl data set
nfl_df.drop(['DSRS', 'League', 'MoV', 'PA', 'PF', 'SRS', 'SoS', 'W-L%', 'OSRS', 'PD'], axis=1, inplace=True)
nfl_df.team = nfl_df.team.astype(str)
#need to regex team names
nfl_df.replace(to_replace=r'[^a-zA-z\s0-9]', value='', regex=True, inplace=True)
nfl_df.set_index('team', inplace=True)
nfl_df.drop(['AFC East','AFC North', 'AFC South', 'AFC West', 'NFC East', 'NFC North', 'NFC South', 'NFC West'], inplace=True)
#map dict
nfl_df.reset_index(inplace=True)
nfl_df.replace(nfl_dictionary, inplace=True)

#-----------------------------------------------------------------------------------------------------------------
#clean mlb data set
mlb_df.team = mlb_df.team.astype(str)
#map dict
mlb_df.replace(mlb_dictionary, inplace=True)

#-----------------------------------------------------------------------------------------------------------------
#clean nba data set
nba_df.drop(['W/L%', 'GB', 'PS/G', 'PA/G', 'SRS'], axis=1, inplace=True)
nba_df.team = nba_df.team.astype(str)
#need to regex team names
nba_df['team'].replace(to_replace=r'[^a-zA-z\s0-9]', value='', regex=True, inplace=True)
nba_df['team'].replace(to_replace=r'[0-9]+$', value='', regex=True, inplace=True)

nba_df.team.str.strip()

nba_df.set_index('team', inplace=True)
nba_df.drop(['Central Division','Southeast Division', 
             'Northwest Division', 'Pacific Division',
             'Southwest Division'], inplace=True)
#map dict
nba_df.reset_index(inplace=True)
nba_df.replace(nba_dictionary, inplace=True)

In [4]:
#nhl_cities cleaning
columns_to_drop_nhl = ['NFL', 'MLB', 'NBA']
nhl_cities.drop(columns = columns_to_drop_nhl, inplace=True)
nhl_cities.dropna(inplace=True)

#mlb_cities cleaning
columns_to_drop_mlb = ['NFL', 'NHL', 'NBA']
mlb_cities.drop(columns = columns_to_drop_mlb, inplace=True)
mlb_cities.dropna(inplace=True)

#nfl_cities cleaning
columns_to_drop_nfl = ['NHL', 'MLB', 'NBA']
nfl_cities.drop(columns = columns_to_drop_nfl, inplace=True)
nfl_cities.dropna(inplace=True)

#nba_cities cleaning
columns_to_drop_nba = ['NFL', 'MLB', 'NHL']
nba_cities.drop(columns = columns_to_drop_nba, inplace=True)
nba_cities.dropna(inplace=True)

In [5]:
#merging DF's
mlb_full = pd.merge(mlb_cities, mlb_df, how='left', left_on='MLB', right_on='team')
nba_full = pd.merge(nba_cities, nba_df, how='left', left_on='NBA', right_on='team')
nfl_full = pd.merge(nfl_cities, nfl_df, how='left', left_on='NFL', right_on='team')
nhl_full = pd.merge(nhl_cities, nhl_df, how='left', left_on='NHL', right_on='team')

In [6]:
#aggregate up to metro area W, L 
#MLB
mlb16 = mlb_full.loc[mlb_full['year'] == 2016]
mlb16.set_index('Metropolitan area', inplace=True)
mlb16.drop(['MLB', 'team', 'W-L%', 'GB', 'year', 'League'], axis=1, inplace=True)
mlb16.astype(float)
mlb16.Population = mlb16.Population.astype(float)
mlb_grouped = mlb16.groupby('Metropolitan area').agg({'W':np.nansum,
                                                      'L':np.nansum,
                                                      'Population':np.nanmean})
ratio = mlb_grouped['W'] / mlb_grouped['L']
mlb_grouped['Win Ratio'] = ratio

#NFL
nfl16 = nfl_full.loc[nfl_full['year'] == 2016]
nfl16.set_index('Metropolitan area', inplace=True)
nfl16.drop(['NFL', 'team', 'T', 'year'], axis=1, inplace=True)
nfl16 = nfl16.astype(float)
nfl16.Population = nfl16.Population.astype(float)
nfl_grouped = nfl16.groupby('Metropolitan area').agg({'Population':np.nanmean,
                                                      'W':sum,
                                                      'L':sum})
ratio = nfl_grouped['W'] / nfl_grouped['L']
nfl_grouped['Win Ratio'] = ratio

#NHL
nhl17 = nhl_full.loc[nhl_full['year'] == 2017]
nhl17.set_index('Metropolitan area', inplace=True)
nhl17.drop(['NHL', 'team', 'year'], axis=1, inplace=True)
nhl17 = nhl17.astype(float)
nhl_grouped = nhl17.groupby('Metropolitan area').agg({'Population':np.nanmean,
                                                      'W':sum,
                                                      'L':sum})
ratio = nhl_grouped['W'] / nhl_grouped['L']
nhl_grouped['Win Ratio'] = ratio

#NBA
nba16 = nba_full.loc[nba_full['year'] == 2016]
nba16.set_index('Metropolitan area', inplace=True)
nba16.drop(['NBA', 'team', 'year', 'League'], axis=1, inplace=True)
nba16 = nba16.astype(float)
nba_grouped = nba16.groupby('Metropolitan area').agg({'Population':np.nanmean,
                                                      'W':sum,
                                                      'L':sum})
ratio = nba_grouped['W'] / nba_grouped['L']
nba_grouped['Win Ratio'] = ratio

In [7]:


def nhl_correlation(): 
    
    population_by_region = nhl_grouped['Population']
    
    win_loss_by_region = nhl_grouped['Win Ratio']
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

def nba_correlation(): 
    
    population_by_region = nba_grouped['Population']
    
    win_loss_by_region = nba_grouped['Win Ratio']
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

def mlb_correlation(): 
    
    population_by_region = mlb_grouped['Population']
    
    win_loss_by_region = mlb_grouped['Win Ratio']
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

def nfl_correlation(): 
    
    population_by_region = nfl_grouped['Population']
    
    win_loss_by_region = nfl_grouped['Win Ratio']
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

def sports_team_performance(): 
    def drop_win_loss(df, name):
        df.drop(['L', 'W', 'Population'], axis = 1, inplace=True)
        df.rename(columns={'Win Ratio': '{x}'.format(x=name)}, inplace=True)
        return df
    drop_win_loss(nba_grouped, 'NBA')
    drop_win_loss(nfl_grouped, 'NFL')
    drop_win_loss(nhl_grouped, 'NHL')
    drop_win_loss(mlb_grouped, 'MLB')
    
    from functools import reduce
    dfs = [nba_grouped, nhl_grouped, nfl_grouped, mlb_grouped]
    df_multiteam = reduce(lambda left, right: left.join(right, how='outer', on='Metropolitan area'), dfs)
    df_multiteam.set_index('Metropolitan area', inplace=True)
    df_multiteam.dropna(thresh=2, inplace=True)
    
    #def get_ttest():
    cols_to_test = ['NBA', 'NFL', 'NHL', 'MLB']
    for x in cols_to_test:
        y = df_multiteam[x]
        a = stats.ttest_rel(y, df_multiteam.MLB, nan_policy='omit')
        b = stats.ttest_rel(y, df_multiteam.NFL, nan_policy='omit') 
        c = stats.ttest_rel(y, df_multiteam.NBA, nan_policy='omit')
        d = stats.ttest_rel(y, df_multiteam.NHL, nan_policy='omit')
        if x == 'NBA':
            ttest_list1 = ['NBA', a[1], b[1], c[1], d[1]]
        elif x == 'NFL':
            ttest_list2 = ['NFL', a[1], b[1], c[1], d[1]]
        elif x == 'NHL':
            ttest_list3 = ['NHL', a[1], b[1], c[1], d[1]]
        else:
            ttest_list4 = ['MLB', a[1], b[1], c[1], d[1]]
    
    data = [ttest_list4, ttest_list2, ttest_list1, ttest_list3]
    ttest_df = pd.DataFrame.from_records(data, columns=['sports', 'MLB', 'NFL', 'NBA', 'NHL'])
    ttest_df.set_index('sports', inplace=True)
    ttest_df.where(ttest_df != str, np.nan, inplace=True)
    p_values = ttest_df
    
    return p_values

In [8]:
nba_correlation()

(-0.053754217618992954, 0.7900153668063975)

In [9]:
mlb_correlation()

(0.22551755660340206, 0.2679871338254316)

In [10]:
nhl_correlation()

(0.2658107448411489, 0.19904993060910978)

In [11]:
nfl_correlation()

(-0.02119413948321257, 0.9131029437960743)

In [12]:
p_values = sports_team_performance()

In [13]:
p_values

,MLB,NFL,NBA,NHL
sports,,,,
MLB,NaN,0.077138,0.374094,0.00212928
NFL,0.077138,NaN,0.877856,0.743751
NBA,0.374094,0.877856,NaN,0.915523
NHL,0.00212928,0.743751,0.915523,NaN
